In [ ]:
from bs4 import BeautifulSoup
import requests
import datetime
#import re
#from pprint import pprint
#import csv
import time
import pandas as pd
#import pickle

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
headers = ['loan value', 'funded percent', 'country', 'loan term', 'loan reason', 'repayment schedule', 'disbursment date', 
           'exchange loss', 'partner agency', 'field partner', 'interest', 'risk rating']
kiva_info = []

In [ ]:
response = requests.get('http://api.kivaws.org/v1/partners.json')

In [ ]:
response.headers

In [ ]:
def pull_data(url):
    response = requests.get(url, headers={'User-Agent': 'Chrome/65.0'})
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    return soup

In [ ]:
def funded_percent(soup):
    try:
        funded = (soup.find(class_='green-bolded inline')).text.strip()[0]
        if (funded == 'F'):
            return 100
        else:
            return int(funded)
    except: pass

In [ ]:
def create_dict(soup):
    #This calculates the percent funded (0-100)
    funded = funded_percent(soup)
    try:
        #This calculates the total requested value of the loan
        loan = int(soup.find(class_='loan-total').text.strip().split('$')[-1])
        
        #This gets the str for the length of the loan term.
        term = (soup.find(id='ac-loan-details-body-right').find(class_='green-bolded').text.strip())

        #This pulls the country
        locale = (soup.find('div', class_='country-image columns small-2'))
        country = str(locale.find('img')).split()[1].split('"')[1]
        
        #Returns the reason for the loan
        reason = soup.find(class_='typeName').text.strip()
        
        #Gets the loan repayment schedule
        schedule = (soup.find(id='ac-loan-details-body-right').find(id='repayment-schedule').text.strip().split(' ')[-1])

        #Returns the date the loan was disbersed
        disbursed = (soup.find(id='ac-loan-details-body-right').find(id='disbursed-date').text.strip().split(' ')[-3:])
        disbursed = datetime.datetime.strptime(" ".join(disbursed),"%B %d, %Y").date()

        #Returns currency exchange loss possibility
        exchange_loss = (soup.find(id='ac-loan-details-body-right').find(id='currency-exchange-loss').text.strip().split(' ')[-1])

        #Returns name of field partner
        field_partner = (soup.find(id='ac-loan-details-body-right').find(id='field-partner').text.strip().split(' '))
        partner = ' '.join(field_partner[0:4]).split(':')[0]
        field_partner = ' '.join(field_partner[-2:])

        #Returns if loan recipient is paying interest
        interest = (soup.find(id='ac-loan-details-body-right').find(id='borrower-paying-interest').text.strip().split(' ')[-1])

        #Calculates the numeric number of stars representing the risk rating
        risk_stars = 0
        for tag in (soup.find(id='ac-loan-details-body-right').find_all(class_='icon icon-star')):
            risk_stars+=1
        for tag in (soup.find(id='ac-loan-details-body-right').find_all(class_='icon icon-half-star')):
            risk_stars +=.5

        kiva_dict = dict(zip(headers, [loan,
                                       funded,
                                       country,
                                       term,
                                       reason,
                                       schedule,
                                       disbursed,
                                       exchange_loss,
                                       partner,
                                       field_partner,
                                       interest,
                                       risk_stars]))
        return kiva_dict
    except:
        pass

In [ ]:
term = (soup.find(id='ac-loan-details-body-right').find(class_='green-bolded').text.strip())

In [ ]:
locale = (soup.find('div', class_='country-image columns small-2'))
country = str(locale.find('img')).split()[1].split('"')[1]

In [ ]:
elements = soup.findAll('img', attrs={'class':''})
for element in elements:
    #print (element['data-src'])
    #print (element['img alt'])
    print(element)

In [ ]:
type(country)

In [ ]:
number = 1507200
while (number <=1507201):
    baseurl = 'https://www.kiva.org/lend/'
    url = baseurl + str(number)
    print (url)
    number+=1
    soup = pull_data(url)
    
    kiva_info.append(create_dict(soup))
    time.sleep(1)
    

In [ ]:
len(kiva_info)

In [ ]:
df3 = pd.DataFrame([x for x in kiva_info if x is not None], columns = headers)

In [ ]:
[x for x in kiva_info if x is not None]

In [ ]:
df.to_pickle('project2.pkl')

In [ ]:
#with open('my_data.pkl', 'wb') as picklefile:
#    pickle.dump(data, picklefile)

In [ ]:
df2= pd.read_pickle('project2.pkl')

In [ ]:
df3['months']= df3['loan term'].str.split().str[0]

In [ ]:
df3.drop(['toss'], axis=1, inplace=True)

In [ ]:
df3.describe()

In [ ]:
#Below uses the data directly from kiva's api

In [ ]:
kiva = pd.read_csv('/Users/travis/Downloads/kiva_ds_csv/loans.csv')

In [ ]:
kiva.drop(['ORIGINAL_LANGUAGE', 'DESCRIPTION', 'DESCRIPTION_TRANSLATED', 'IMAGE_ID', 'VIDEO_ID', 'TOWN_NAME', 'TAGS', 'BORROWER_NAMES', 'BORROWER_PICTURED', 'LOAN_USE', 'COUNTRY_CODE'], axis=1, inplace=True)

In [ ]:
#kiva.drop(['LOAN_USE', 'COUNTRY_CODE'], axis=1, inplace= True)
#kiva.drop(['COUNTRY_CODE'], axis=1, inplace=True)

In [ ]:
kiva.describe()

In [ ]:
kiva[(kiva['LOAN_AMOUNT'] == 100000)]

In [ ]:
kiva.info()

In [ ]:
kiva.sample(50)

In [ ]:
kiva[(kiva['LOAN_AMOUNT'] > kiva['FUNDED_AMOUNT']) & (kiva['DISTRIBUTION_MODEL'] == 'direct')]

In [ ]:
kiva[kiva['DISTRIBUTION_MODEL'] == 'direct']

In [ ]:
kiva['COUNTRY_NAME'].value_counts()

In [ ]:
70000/1420000

In [ ]:
f, ax = plt.subplots(figsize=(12,12))
sns.countplot(x='COUNTRY_NAME', data=kiva)

In [ ]:
kiva.to_pickle('kiva.pkl')